# Training Algorithm

> Set up the basic training loop

In [ ]:
#| default_exp training

In [ ]:
#| export
from typing import Sequence

import lightning as L
import torch
from pydantic import BaseModel

from slow_diffusion.model import Unet

In [ ]:
# |exports
class UnetConfig(BaseModel):
    """Model configuation"""

    # Number of channels in a {Up,Down}block
    nfs: Sequence[int]

    # Number of sub-blocks in {Up,Down}block. Should have 1 more entry than `nfs`
    n_blocks: Sequence[int]

    # Color channels, or however many dimensions in the VAE bottleneck space
    color_channels: int

In [ ]:
# |exports
class TrainingConfig(BaseModel):
    """Training configuation"""

    lr: float

In [ ]:
# |exports
class Config(BaseModel):
    """Top-level configuation"""

    unet_config: UnetConfig
    training_config: TrainingConfig

In [ ]:
# |exports
class UnetLightning(L.LightningModule):
    def __init__(self, config: Config):
        super().__init__()
        self.unet = Unet(**config.unet_config.dict())
        self.save_hyperparameters(config.dict())
        self.loss_fn = torch.nn.MSELoss()

    def step(self, batch):
        (x_t, t), epsilon = learn.batch
        preds = self.unet(x_t, t)
        return self.loss_fn(preds, epsilon)

    def training_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log("trn_loss", loss, sync_dist=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        self.log("tst_loss", loss, sync_dist=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.lr)
        if self.config.one_cycle_scheduler is False:
            return optimizer
        else:
            return {
                "optimizer": optimizer,
                "lr_scheduler": {
                    "scheduler": torch.optim.lr_scheduler.OneCycleLR(
                        optimizer,
                        max_lr=self.config.lr,
                        total_steps=self.trainer.estimated_stepping_batches,
                        pct_start=self.config.one_cycle_config.pct_start,
                        div_factor=self.config.one_cycle_config.div_factor,
                        final_div_factor=self.config.one_cycle_config.final_div_factor,
                    ),
                    "interval": "step",
                    "frequency": 1,  # Update the LR every step
                    "monitor": "tst_loss",  # Not relevant for OneCycleLR
                    "strict": True,  # Doesn't need to be strict because the monitor is irrelevant
                },
            }

In [ ]:
cfg = Config(
    unet_config=UnetConfig(
        nfs=(224, 448, 672, 896),
        n_blocks=(3, 2, 2, 1, 1),
        color_channels=3,
    ),
    training_config=TrainingConfig(lr=4e-3),
)
unet = UnetLightning(cfg)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()